In [1]:
%display latex
import sage.misc.banner; sage.misc.banner.banner()

┌────────────────────────────────────────────────────────────────────┐
│ SageMath version 10.7, Release Date: 2025-08-09                    │
│ Using Python 3.12.11. Type "help()" for help.                      │
└────────────────────────────────────────────────────────────────────┘


This code can be used to compute $\dim({\bf U}_A[p^n])$, where: 
* $A$ is an abelian variety of dimension $g$ over the finite field $K=\mathbb{F}_q$ of characteristic $p>2$. 
* $n\ge1$ is an integer.
* ${\bf U}_A$ is the quasi-algebraic unipotent group whose group of $\overline{k}$-points is the non-$p$-divisible part of $\mathrm{Br}(A_{\overline{k}})$.
  
Let $M$ be the Dieudonné module of $A$, $F,V$ the Frobenius and Verschiebung. We define a group scheme $\tilde{U}_A$ whose $R$-points are given by
$$\tilde{U}_A(R)=\{f\in\mathrm{Hom}_{W_n(R)}(W_n(R)\otimes_{W(k)}M^{\vee}/p^n,W_n(R)\otimes_{W(k)}M/p^n)^{\textnormal{skew}}\text{ s.t. }F^{\vee}\textnormal{Frob}(A)=AF\}$$
This is a finite type group scheme over $k$ whose $\overline{k}$-points are $${\bf U}_A(\overline{k})=\mathrm{Hom}_{\mathbb{E}_{\overline{k}}}(M^{\vee}/p^n,M/p^n)^{\textnormal{skew}},$$ so its dimension is indeed equal to $\dim({\bf U}_A[p^n])$.

In [2]:
g = 4
p = 3
q = p^2
K = GF(q)
n = 2

The group $\tilde{U}_A$ is a subgroup scheme of the skew-symmetric elements of $\underline{\mathrm{End}}_k(\mathbb{W}_n^{2g})$, which is a ring scheme with coordinate ring $k[x_{ijk}|1\le i,j\le 2g,1\le k\le n]$. The coordinates should be thought as parametrizing matrices whose entry $(i,j)$ is given by the Witt vector $(x_{ij1},\dots,x_{ijn})$. The sum and product are given by the Witt vector sum and product laws. By design all elements in our subgroup are skew-symmetric, so it will define an ideal of the smaller ring 
$$R=k[x_{ijk}|1\le i\le 2g,1\le j<i, 1\le k\le n].$$

We define $R$, the Witt vectors of $R$, and the Frobenius endomorphism on the Witt vectors.

In [3]:
R = PolynomialRing(K,'x',n*g*(2*g-1))

finv = K.frobenius_endomorphism(-1)

# source code in: sage/rings/padics/witt_vector_ring.py
WR = WittVectorRing(R, p=p, prec=n, algorithm='finotti')
WR0,WR1,WRp = WR.zero(),WR.one(),p*WR.one()

def wittFrob(w,W):
    t = list(w.coordinates())
    for i in range(len(t)):
        t[i]=t[i]^p
    return W(t)

In order to make the computations with matrices of Witt vectors more efficient, we implement some basic methods to work with sparse matrices.

In [4]:
# convert a square sparse column array into a matrix
def CSCtoMat(ring,csc):
    m = matrix(ring,len(csc),len(csc))
    for i in range(len(csc)):
        for j,v in csc[i]:
            m[j,i]=v
    return m

# multiply a matrix (left) with a sparse column array (right)
def mul(mat,csc):
    Nr,Nc = mat.nrows(),len(csc)
    retval = matrix(mat.base_ring(),Nr,Nc)
    for i in range(Nr):
        for j in range(Nc):
            for k,v in csc[j]:
                retval[i,j] += v*(mat[i,k])
    return retval

# direct sum of square sparse column arrays
def dirSumCSC(csc1,csc2):
    csc = csc1[:]
    for col in csc2:
        csc.append([(el[0]+len(csc1),el[1]) for el in col])
    return csc

Suppose that we are given $F,V$ as matrices in a $W_n(k)$-basis of $M/p^n$. Then $F^{\vee}=\mathrm{Frob}(V^t)$ is the matrix of the Frobenius of $M^{\vee}$ in the dual basis. The group scheme we compute parametrises matrices in $\mathrm{End}(\mathbb{W}(k)^{\oplus 2g})$ which satisfy $$A^t=-A;\quad\quad DF*\mathrm{Frob}(A)=A*F$$

In [5]:
# define the matrix of indeterminates
A = matrix(WR,2*g,2*g)
idx = 0
for i in range(2*g):
    for j in range(i):
        A[i,j] = WR(R.gens()[n*idx:n*(idx+1)])
        A[j,i] = -A[i,j]
        idx += 1
del idx
Apt = A.apply_map(lambda x:wittFrob(x,WR)).transpose()

In [6]:
# calculate the dimension given F and DF transpose
def dimension(g,n,F,DFt):
    S = (mul(Apt,DFt)).transpose()-mul(A,F)
    gens = [S[i][j].coordinates()[k] for i in range(2*g) for j in range(2*g) for k in range(n)]
    return ideal(gens).dimension()

## First example: g=4, a-number 1, p^2-torsion

This next function builds the matrices $F,V$ for a finite set of supersingular Dieudonné modules with $a$-number $1$ and $g=4$. The Dieudonné module of an abelian variety with $p$-rank $0$ and $a$-number $1$ is isomorphic to $\mathbb{E}/f$, where $$f=F^g+a_1F^{g-1}+\dots+a_g+b_1V+\dots+b_{g-1}V^{g-1}+V^g$$ The supersingularity condition is equivalent to $v(a_i)\ge i/2$ and $v(b_i)\ge (g-i)/2$. Therefore, to compute the $p^2$-torsion we only care about $a_1,a_2,b_{g-2},b_{g-1}$. 

The input is four numbers $l1,l2,l3,l4$. Then we take $a$ to be a multiplicative generator of $K$, and set $$a_1=a^{l1},a_2=a^{l2},b_{g-2}=a^{l3},b_{g-1}=a^{l4}$$
We also allow these to be zero. We return the matrices of $F,V$ in the basis $F^{g-1},\dots,1,V,\dots,V^g$.

In [7]:
# define parameters for building F and DF
a = K.multiplicative_generator()
pows = [R.zero()]+[R.one()*a^l for l in range(q-1)]
pars = [WR([pows[0],pows[l]]) for l in range(q)]

# build F and DF transpose as sparse column arrays
def buildF(l1,l2,l3,l4):
    F = [[((i-1)%(2*g),WR1)] for i in range(g)]+[[((i-1)%(2*g),WRp)] for i in range(g,2*g)]
    F[0] += [(0,pars[l1]),(1,pars[l2]),(2*g-3,pars[l3]),(2*g-2,pars[l4])]
    #
    DFt = [[((i+1)%(2*g),WRp)] for i in range(g-1)]+[[((i+1)%(2*g),WR1)] for i in range(g-1,2*g-1)]+[[(0,WRp)]]
    DFt[2*g-1] += [(2*g-2,-pars[l3]),(2*g-1,-pars[l4])]
    #
    return F,DFt

We iterate through all possible values of $l1,l2,l3,l4\in[0,q-1]$. According to Question 5.4 in our paper, we expect that the dimension of the $p^2$-torsion is $5$. If we find that the computed dimension for a certain set of parameters is not equal to $5$, we display the eccentric dimension and the set of parameters it comes from. The result of the following simulation is that for every supersingular abelian fourfold with $a$-number $1$, such that $M_A$ is defined over $F_{9}$, the dimension of ${\bf U}_A[p^2]$ is $5$. This means that for these supersingular abelian fourfolds, the isogeny type of ${\bf U}_A$ is $\mathbb{W}_3\times\mathbb{W}_2\times\mathbb{W}_1$.

In [8]:
display('Start of search ({} iterations)'.format((q)^4))

import time
start_time = time.time()
found = []

# !!! parallelize the loop?
for l in xmrange([q,q,q,q]):
    # print progress
    if l[2]+l[3] == 0:
        print('Starting l1={} l2={} ({:0.2f}% done, {:0.1f}s)'.format( l[0], l[1], (l[1]/q^2+l[0]/q)*100.0, time.time()-start_time ))
    # check dimension
    found_dim = dimension(g,n,*buildF(*l))
    if found_dim != 5:
        display(table(['Dimension {}≠{} (expected) for '.format(found_dim,expected_dim)]+l))
        found.append((found_dim,l))

display('End of search, found {}'.format(len(found)))

'Start of search (6561 iterations)'

Starting l1=0 l2=0 (0.00% done, 0.0s)
Starting l1=0 l2=1 (1.23% done, 0.7s)
Starting l1=0 l2=2 (2.47% done, 1.4s)
Starting l1=0 l2=3 (3.70% done, 2.0s)
Starting l1=0 l2=4 (4.94% done, 2.7s)
Starting l1=0 l2=5 (6.17% done, 3.5s)
Starting l1=0 l2=6 (7.41% done, 4.2s)
Starting l1=0 l2=7 (8.64% done, 4.8s)
Starting l1=0 l2=8 (9.88% done, 5.5s)
Starting l1=1 l2=0 (11.11% done, 6.1s)
Starting l1=1 l2=1 (12.35% done, 6.8s)
Starting l1=1 l2=2 (13.58% done, 7.5s)
Starting l1=1 l2=3 (14.81% done, 8.2s)
Starting l1=1 l2=4 (16.05% done, 8.8s)
Starting l1=1 l2=5 (17.28% done, 9.5s)
Starting l1=1 l2=6 (18.52% done, 10.2s)
Starting l1=1 l2=7 (19.75% done, 10.9s)
Starting l1=1 l2=8 (20.99% done, 11.6s)
Starting l1=2 l2=0 (22.22% done, 12.2s)
Starting l1=2 l2=1 (23.46% done, 12.9s)
Starting l1=2 l2=2 (24.69% done, 13.6s)
Starting l1=2 l2=3 (25.93% done, 14.3s)
Starting l1=2 l2=4 (27.16% done, 15.0s)
Starting l1=2 l2=5 (28.40% done, 15.7s)
Starting l1=2 l2=6 (29.63% done, 16.5s)
Starting l1=2 l2=7 (30.8

'End of search, found 0'